In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import collections
import re
import pprint as pp
import numpy as np
import collections

import multiprocessing as mp
from multiprocessing.pool import ThreadPool

import math
import gzip
import pickle as pkl
from datetime import datetime
import matplotlib.pyplot as plt

import fonctions
import itertools

from os import listdir
from os.path import isfile, join

import random
random.seed(0)

Using TensorFlow backend.


In [2]:
def trend_processing(x):
    if type(x) is str:
        return re.split(r'\t+', x)
    return float('Nan')

def get_trends(directory, doc_name, init):
    
    all_features = ["text_tokens", "hashtags", "tweet_id", 
                    "present_media", "present_links", 
                    "present_domains", "tweet_type","language", 
                    "tweet_timestamp", "engaged_with_user_id",
                    "engaged_with_user_follower_count", "engaged_with_user_following_count", 
                    "engaged_with_user_is_verified", "engaged_with_user_account_creation",
                    "engaging_user_id", "engaging_user_follower_count", 
                    "engaging_user_following_count", "engaging_user_is_verified",
                    "engaging_user_account_creation", "engagee_follows_engager"]
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep='\x01', header=None)
    
    if init:
        labels = ['reply_timestamp','retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp']
        all_variables = all_features + labels
        df.columns = all_variables
    else:
         df.columns = all_features
        
    df['tweet_timestamp']=[ str(datetime.utcfromtimestamp(int(date)))[0:13] for date in df['tweet_timestamp'] ]    
    df = df.filter(['hashtags','tweet_timestamp','tweet_id'],axis=1)
    
    return df


def hashtags_on_chunk(directory, chunk, chunk_id, init):

    hashtag_presence = {}

    iteration=1
    for batch_file in chunk:
        
        df = get_trends(directory,batch_file,init)
        df = df [ df['hashtags'].isna()==False ]
        df = df.drop_duplicates()
        df['hashtags'] = [ trend_processing(x) for x in df['hashtags'] ]

        hashtags = list( itertools.chain.from_iterable(df.hashtags) ) 

        tweet_id=[ [(x1,x3)] if len(x2)==1 else [ (x1,x3) ]*len(x2)  for x1,x2,x3 in zip(df.tweet_id, df.hashtags, df.tweet_timestamp) ]
        tweet_id = np.array( list( itertools.chain.from_iterable(tweet_id) ) )
        df2 = pd.DataFrame({'hashtags':hashtags, 'tweet_id':tweet_id[:,0], 'tweet_timestamp':tweet_id[:,1] })
        dictio = reformater(df2)

        select = { k:v for k,v in dictio.items() if k not in hashtag_presence.keys()  }
        reste = { k:v for k,v in dictio.items() if k in hashtag_presence.keys()  }
        
        hashtag_presence.update( select )
        { update_links(hashtag_presence, k, v) for k,v in reste.items() }
        
        print(iteration)
        iteration=iteration+1

    print('cutting and saving...')
    
    hashtag_presence = {k:sorting(v) for k,v in hashtag_presence.items() }
    if init==True:

        with gzip.open('/home/maxime/Desktop/RecSys2020/trends/hashtag_presence_{}.pkl.gz'.format(chunk_id), 'wb') as f:
            pkl.dump(hashtag_presence,f)
            
    else: 
        
        with gzip.open('/home/maxime/Desktop/RecSys2020/trends/update1_hashtag_presence_{}.pkl.gz'.format(chunk_id), 'wb') as f:
            pkl.dump(hashtag_presence,f)
            
    return True

def reformater(df):
    dictio=collections.defaultdict(list)
    for x1,x2,x3 in zip(df.hashtags, df.tweet_id, df.tweet_timestamp):
        if x1 in dictio.keys():
            dictio[ x1 ].append( (x2, x3) )
        else:
            dictio[ x1 ]= [ (x2,x3) ]
            
    return dictio

def update_links(links_rank, k, v):
    links_rank[k].extend( v )

def sorting(v):
    unique = set(v)
    res = sorted(unique, key=lambda x: datetime.strptime(str(x[1]), '%Y-%m-%d %H') )
    return res
    

In [10]:
%%time

batch_path='/home/maxime/Desktop/RecSys2020/data/batches'
batch_list = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]
chunks = fonctions.chunkIt(batch_list, 8)
directory = '/home/maxime/Desktop/RecSys2020/data/batches/'

if __name__ == '__main__':
    
    # Setup a list of processes that we want to run
    processes = [ mp.Process(target=hashtags_on_chunk, args=(directory, chunk, chunk_id, True) ) for chunk_id, chunk in zip( [0],[chunks[0]  ] ) ]

    # Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join() 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
%%time

batch_path='/home/maxime/Desktop/RecSys2020/data/test'
batch_list = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]
chunks = fonctions.chunkIt(batch_list, 8)
directory = '/home/maxime/Desktop/RecSys2020/data/test/'

if __name__ == '__main__':
    
    # Setup a list of processes that we want to run
    processes = [ mp.Process(target=hashtags_on_chunk, args=(directory, chunk, chunk_id, False) ) for chunk_id, chunk in enumerate(chunks) ]

    # Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join() 

1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
10
9
10
10
10
10
10
10
11
11
10
11
11
11
12
11
12
11
12
12
11
12
13
13
12
13
12
13
12
13
14
13
13
14
14
14
13
15
14
14
14
14
15
15
15
15
16
16
15
15
15
16
16
16
17
17
16
16
16
17
17
17
18
18
17
17
17
18
18
18
19
19
18
18
18
19
19
19
20
19
20
19
20
20
19
21
20
21
20
20
21
21
20
21
22
22
21
21
21
22
22
23
22
23
22
22
23
22
23
23
24
24
23
23
23
24
24
25
25
24
24
24
25
24
26
26
25
25
25
25
26
25
27
27
26
26
26
26
27
26
28
28
27
27
27
27
27
28
29
28
28
29
28
28
29
28
30
29
30
29
29
29
29
30
31
30
31
30
30
30
31
30
32
31
32
31
31
31
32
31
33
32
33
33
32
32
32
32
34
33
33
34
34
33
35
33
34
35
33
35
34
34
36
35
34
36
36
35
34
35
37
36
35
37
36
37
35
38
36
36
37
38
37
38
36
37
37
39
38
37
40
39
38
39
39
38
38
40
41
40
38
39
40
39
39
41
42
41
40
39
41
40
40
42
42
43
40
41
42
41
43
41
42
41
43
42
43
44
42
44
42
43
44
45
43
43
44
45
45
46
43
44
44
46
45
46
4